* [地理/逆地理编码](https://lbs.amap.com/api/webservice/guide/api/georegeo)
* [行政区域查询](https://lbs.amap.com/api/webservice/guide/api/district)
* [路径规划](https://lbs.amap.com/api/webservice/guide/api/direction)

In [27]:
import requests
import json
import pandas as pd
from pandas import DataFrame
from datetime import datetime

def get_coord(address):   
    '''解析地址，获取经纬度'''
    result = {}
    url = 'https://restapi.amap.com/v3/geocode/geo?parameters'    
    params = {'key':'66ba580836eb10be0deed54e77633cf5',                  
              'address': address,                        
              'city': 'hangzhou',
             }         
    try:
        res = requests.get(url,params)        
        jd = json.loads(res.text)
        coord = jd['geocodes'][0]['location']
        return coord    
    except:
        print("未获取到经纬度！")

def get_driving_distance(company_add):
    '''获取家和公司的距离'''
    home_add = "杭州-余杭区-富力十号院"
    home_coord = get_coord(home_add)
    
    company_coord = get_coord(company_add)
    url = 'https://restapi.amap.com/v3/direction/driving?parameters'            
    params = {'key':'66ba580836eb10be0deed54e77633cf5', 
              'origin': home_coord,
              'destination': company_coord,      
            }
    res = requests.get(url, params)
    jd = json.loads(res.text)
    distance = jd['route']['paths'][0]['distance']
    return distance

def main():
    df = pd.read_excel('D:/liepin/liepin_20180802.xlsx')
    address = df['公司地址'].tolist()
    dct = {}
    for add in address:
        distance = get_driving_distance(add)
        dct[add] = distance
    data = pd.DataFrame.from_dict(dct,orient='index')
    data = data.reset_index()
    data.columns =['公司地址','离家驾车距离']
    data['离家驾车距离'] = data['离家驾车距离'].astype('int')
    new_df = df.merge(data,on='公司地址',how='left')
    new_df.to_excel('D:/lagou/lagou_with_distance_{}.xlsx'.format(datetime.now().strftime('%Y%m%d')),index=False,encoding='gbk')

if __name__ == '__main__':
    main()